In [50]:
#Importing the Libraries
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

In [51]:
data_test = pd.read_csv("test.csv")
print("Number of datapoints:", len(data_test))
data_test.head()

Number of datapoints: 418


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [52]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [53]:
#Found Null Values on Age, Fare and Cabin
data_test.isnull().sum()

missing_percentage = (data_test.isnull().sum() / len(data_test))*100
print(missing_percentage)

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.574163
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.239234
Cabin          78.229665
Embarked        0.000000
dtype: float64


Missing Completely at Random (MCAR)
- No reason for missing values
Missing at Random (MAR)
- Missing values are not random
Missing Not at Random (MNAR)
- Missing values depend on unobserved data

In [54]:
meanAge = data_test['Age'].mean()
data_test['Age'].fillna(value=meanAge, inplace=True)

meanFare = data_test['Fare'].mean()
data_test['Fare'].fillna(value=meanFare, inplace=True)

data_test = data_test.drop('Cabin',axis=1)

In [55]:
data_test.isnull().sum()

missing_percentage = (data_test.isnull().sum() / len(data_test))*100
print(missing_percentage)

PassengerId    0.0
Pclass         0.0
Name           0.0
Sex            0.0
Age            0.0
SibSp          0.0
Parch          0.0
Ticket         0.0
Fare           0.0
Embarked       0.0
dtype: float64


In [56]:
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [57]:
#Feature Engineering
data_test['FamilyCount'] = data_test['Parch'] + data_test['SibSp']

for index, row in data_test.iterrows():
    if(row['FamilyCount'] == 0):
        data_test['FamilySize'] = 'Alone'
    elif(row['FamilyCount'] >= 1 and row['FamilyCount'] < 3 ):
        data_test['FamilySize'] = 'Small'
    elif(row['FamilyCount'] >= 3 and row['FamilyCount'] < 5 ):
        data_test['FamilySize'] = 'Medium'
    elif(row['FamilyCount'] >= 5 and row['FamilyCount'] < 7 ):
        data_test['FamilySize'] = 'Large'


In [58]:
data_test.head(30)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,FamilyCount,FamilySize
0,892,3,"Kelly, Mr. James",male,34.50000,0,0,330911,7.8292,Q,0,Small
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00000,1,0,363272,7.0000,S,1,Small
2,894,2,"Myles, Mr. Thomas Francis",male,62.00000,0,0,240276,9.6875,Q,0,Small
3,895,3,"Wirz, Mr. Albert",male,27.00000,0,0,315154,8.6625,S,0,Small
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00000,1,1,3101298,12.2875,S,2,Small
5,897,3,"Svensson, Mr. Johan Cervin",male,14.00000,0,0,7538,9.2250,S,0,Small
6,898,3,"Connolly, Miss. Kate",female,30.00000,0,0,330972,7.6292,Q,0,Small
7,899,2,"Caldwell, Mr. Albert Francis",male,26.00000,1,1,248738,29.0000,S,2,Small
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.00000,0,0,2657,7.2292,C,0,Small
9,901,3,"Davies, Mr. John Samuel",male,21.00000,2,0,A/4 48871,24.1500,S,2,Small
